In [66]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import visualize as v

from tradebook import Trade, TradeBook
from functions import debug, log

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Visualize Data

In [4]:
hb_ob_data = pd.read_csv("orderbook_hb.csv", index_col=0)
bnc_ob_data = pd.read_csv("orderbook_bnc.csv", index_col=0)

In [64]:
hb_bnc = pd.concat([hb_ob_data["b1"], (hb_ob_data["a1"] + hb_ob_data["b1"])/2, hb_ob_data["a1"], bnc_ob_data["b1"], (bnc_ob_data["a1"] + bnc_ob_data["b1"])/2, bnc_ob_data["a1"]], axis=1)
hb_bnc.columns = ["HB Best Bid", "HB Mid", "HB Best Offer", "BNC Best Bid", "BNC Mid", "BNC Best Offer"]
hb_bnc["HB/BNC Bid"] = hb_bnc["HB Best Bid"] - hb_bnc["BNC Best Offer"]
hb_bnc["HB/BNC Offer"] = hb_bnc["HB Best Offer"] - hb_bnc["BNC Best Bid"]
hb_bnc["HB BV"] = hb_ob_data["bv1"]
hb_bnc["HB AV"] = hb_ob_data["av1"]
hb_bnc["BNC BV"] = bnc_ob_data["bv1"]
hb_bnc["BNC AV"] = bnc_ob_data["av1"]
hb_bnc

,HB Best Bid,HB Mid,HB Best Offer,BNC Best Bid,BNC Mid,BNC Best Offer,HB/BNC Bid,HB/BNC Offer,HB BV,HB AV,BNC BV,BNC AV
timestamp,,,,,,,,,,,,
2021-08-20 00:00:00,27.0999,27.09995,27.1000,27.095,27.0980,27.101,-0.0011,0.0050,10.67,83.08,988.450,21.079
2021-08-20 00:00:01,27.0920,27.09335,27.0947,27.095,27.0975,27.100,-0.0080,-0.0003,22.00,2.99,964.450,21.079
2021-08-20 00:00:02,27.0923,27.09305,27.0938,27.095,27.0955,27.096,-0.0037,-0.0012,7.97,32.18,462.036,20.000
2021-08-20 00:00:03,27.0923,27.09235,27.0924,27.090,27.0950,27.100,-0.0077,0.0024,6.36,33.83,23.642,1.079
2021-08-20 00:00:04,27.0923,27.09235,27.0924,27.093,27.0970,27.101,-0.0087,-0.0006,5.36,33.83,1.079,5.265
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-20 01:59:55,27.2561,27.25990,27.2637,27.260,27.2605,27.261,-0.0049,0.0037,17.64,36.68,208.367,11.477
2021-08-20 01:59:56,27.2561,27.25990,27.2637,27.260,27.2605,27.261,-0.0049,0.0037,17.64,36.68,92.914,11.477
2021-08-20 01:59:57,27.2477,27.25275,27.2578,27.248,27.2505,27.253,-0.0053,0.0098,95.00,36.00,125.095,0.684


In [45]:
v.plot_line(hb_bnc[["HB Best Bid", "HB Best Offer", "BNC Best Bid", "BNC Best Offer"]])

In [46]:
v.plot_line(hb_bnc[["HB Best Bid", "HB Best Offer", "BNC Best Bid", "BNC Best Offer"]])

#### Trade Strategy

Let $B_{i,t}, A_{i,t}, B_{j,t}, A_{j,t}$ be the bid and ask price on exchange $i$ and $j$ at timestamp $t$ respectively

Let $B_{s,t}, A_{s,t}$ be the bid and ask price for the cross-exchange spread at timestamp $t$, where 
$$
B_{s,t} = B_{j,t} - A_{i,t}
$$
$$
A_{s,t} = A_{j,t} - B_{i,t}
$$

Then there exists an arbitrage opportunity whenever $B_{s,t} > 0$ or $A_{s,t} < 0$ because the theoretical fair value of the spread is zero by the Law of One Price, assuming no cross-exchange market impact and zero commissions and other costs.

Note that we will only be dealing with L1 layer of orderbook data, because we are unable to simulate accurately the actions of other market participants after clearing L1 bids or offers on each exchange.

We define our trading strategy as follows

Trade parameters:
- $c$ - minimum absolute deviation from 0 to trigger trade entry
- $\pi$ - minimum profit target per trade to trigger trade exit
- $P_\tau$ - entry price
- $P_T$ - exit price
- $V_{B, i, \tau}$ - volume of bid on exchange $i$ on entry
- $V_{A, i, \tau}$ - volume of offer on exchange $i$ on entry
- $V_{B, j, \tau}$ - volume of bid on exchange $j$ on entry
- $V_{A, j, \tau}$ - volume of offer on exchange $j$ on entry

Whenever 
- $A_{s,t} = P_\tau < -c$, then lift $A_{j,t}$ and hit $B_{i,t}$ in $min(V_{A, j, \tau}, V_{B, i, \tau})$ lots each, OR 
- $B_{s,t} = P_\tau > c$, then hit $B_{j,t}$ and lift $A_{i,t}$ in $min(V_{A, i, \tau}, V_{B, j, \tau})$ lots each

Then wait till $|P_T - P_\tau| >= \pi$, then exit the trade.


Note that by buying and selling the same asset across exchanges, we don't bear any market (price) risk, but we are still subject to liquidity risk if the volume we can sell (or buy) at exit is less than the volume we bought (or sold) at entry. If such a case happens, we liquidate as much of the position as possible by exiting BOTH legs of the spread. We subsequently look out for more opportunities to exit the spread at another time when more volume allows us to do so at our profit target level.

In [114]:
c = 0
pi = 5 * 0.0001
hb_symbol = "LINKUSDT.HB"
bnc_symbol = "LINKUSDT.BNC"
tb = TradeBook()
report = []
# Iterate through the order book data
for i, (ts, row) in enumerate(hb_bnc.iterrows()):
    A_st = row["HB/BNC Offer"]
    V_Ajt = row["HB AV"]
    V_Bit = row["BNC BV"]
    V_Ait = row["HB BV"]
    V_Bjt = row["BNC AV"]
    B_st = row["HB/BNC Bid"]
    BV = min(V_Ait, V_Bjt)
    AV = min(V_Ajt, V_Bit)

    if i % 5 == 0:
        print("")

    # Check for all open trades if any of them have met exit level condition
    BV, AV = tb.check_exit_trigger(ts=ts, bid=B_st, offer=A_st, bv=BV, av=AV)

    # Check if trade entry level condition is met
    if A_st < -c:
        # If so, open a new trade by recording entry level, ts, volume, and exit level condition
        tb.add_trade(entry_ts=ts, 
                     entry=A_st,
                     entry_leg1=row["HB Best Offer"],
                     entry_leg2=row["BNC Best Bid"],
                     open_vol=AV, 
                     exit_target=pi, 
                     symbol=f"{hb_symbol}/{bnc_symbol}")
        
        log(f"Bought HB, Sold BNC at {A_st} on {ts}, x{AV}")

    if B_st > c:
        tb.add_trade(entry_ts=ts, 
                     entry=B_st,
                     entry_leg1=row["HB Best Bid"],
                     entry_leg2=row["BNC Best Offer"],
                     open_vol= -1 * BV, 
                     exit_target=pi, 
                     symbol=f"{bnc_symbol}/{hb_symbol}")
        
        log(f"Sold HB, Bought BNC at {B_st} on {ts}, x{BV}")

    # Mark all open trades against mid price and calculate open and realised pnl for the day
    spread_settlement = (A_st + B_st) / 2
    tb.eod(ts, spread_settlement)


Bought HB, Sold BNC at -0.0002999999999993008 on 2021-08-20 00:00:01, x2.99
Bought HB, Sold BNC at -0.0011999999999972033 on 2021-08-20 00:00:02, x32.18
Bought HB, Sold BNC at -0.0005999999999986017 on 2021-08-20 00:00:04, x1.079

Closed trade #1 at 0.0006000000000021544 on 2021-08-20 00:00:08, remaining: x2.62
Sold HB, Bought BNC at 0.0006000000000021544 on 2021-08-20 00:00:08, x0


Bought HB, Sold BNC at -0.005400000000001626 on 2021-08-20 00:00:15, x20.0
Bought HB, Sold BNC at -0.008900000000000574 on 2021-08-20 00:00:17, x0.738
Bought HB, Sold BNC at -0.001300000000000523 on 2021-08-20 00:00:18, x24.0



Bought HB, Sold BNC at -0.002400000000001512 on 2021-08-20 00:00:31, x17.59
Bought HB, Sold BNC at -0.004799999999999471 on 2021-08-20 00:00:33, x6.431148750331966

Bought HB, Sold BNC at -0.00140000000000029 on 2021-08-20 00:00:39, x3.697

Bought HB, Sold BNC at -0.0019000000000026773 on 2021-08-20 00:00:42, x11.84

Bought HB, Sold BNC at -0.0020999999999986585 on 2021-08-20 00:0

In [124]:
mo_report = pd.DataFrame(tb.reports)
mo_report.set_index('timestamp', inplace=True)
mo_report["Return"] = (mo_report["P&L"] - mo_report["P&L"].shift(1)) / (mo_report["P&L"].shift(1))
mo_report

,P&L,Return
timestamp,,
2021-08-20 00:00:00,0.000000,NaN
2021-08-20 00:00:01,-0.011512,-inf
2021-08-20 00:00:02,-0.046654,3.052773
2021-08-20 00:00:03,-0.053688,0.150771
2021-08-20 00:00:04,-0.128397,1.391571
...,...,...
2021-08-20 01:59:55,137.336396,0.000000
2021-08-20 01:59:56,137.336396,0.000000
2021-08-20 01:59:57,137.361117,0.000180


In [136]:
v.plot_line(mo_report["P&L"], title="Mark-to-Market of Arbitrage Strategy per timestamp")

Given that this is a pure arbitrage strategy, risk metrics such as sharpe ratio are not as relevant because it cannot be compared against other non-arb strategies. The daily mtm down swings are simply due to the fact that we mark against the mid price. At some point buying above or selling below fair value will eventually realise a profit as long as appropriate entry threshold is selected.